In [1]:
bronze_worldbank_gdp = spark.sql("SELECT * FROM Itransition_project.dbo.bronze_worldbank_gdp LIMIT 1000")
bronze_worldbank_gdp = bronze_worldbank_gdp.toPandas()
bronze_worldbank_gdp

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 3, Finished, Available, Finished)

,year,gdp_usd
0,2024,28750956130731
1,2023,27292170793214
2,2022,25604848907611
3,2021,23315080560000
4,2020,21060473613000
...,...,...
60,1964,684144620690
61,1963,637058551724
62,1962,603639413793
63,1961,561940310345


In [2]:
bronze_ecb_fx_rates = spark.sql("SELECT * FROM Itransition_project.dbo.bronze_ecb_fx_rates")
bronze_ecb_fx_rates = bronze_ecb_fx_rates.toPandas()
bronze_ecb_fx_rates

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 4, Finished, Available, Finished)

,date,usd_eur
0,2025-12-31,1.1750
1,2025-12-30,1.1757
2,2025-12-29,1.1766
3,2025-12-24,1.1787
4,2025-12-23,1.1786
...,...,...
6970,2000-12-25,NaN
6971,2000-05-01,NaN
6972,2000-04-24,NaN
6973,2000-04-21,NaN


In [3]:
silver_fx_daily = bronze_ecb_fx_rates.dropna(subset=["usd_eur"])
silver_fx_daily

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 5, Finished, Available, Finished)

,date,usd_eur
0,2025-12-31,1.1750
1,2025-12-30,1.1757
2,2025-12-29,1.1766
3,2025-12-24,1.1787
4,2025-12-23,1.1786
...,...,...
6908,1999-01-08,1.1659
6909,1999-01-07,1.1632
6910,1999-01-06,1.1743
6911,1999-01-05,1.1790


In [4]:
silver_taxi_daily_zone = spark.sql("SELECT * FROM Itransition_project.dbo.silver_taxi_daily_zone")
silver_taxi_daily_zone = silver_taxi_daily_zone.toPandas()
silver_taxi_daily_zone

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 6, Finished, Available, Finished)

,date,Borough_p,PULocationID,trips_count,total_revenue_usd,avg_total_amount_usd,avg_trip_distance,avg_duration_min
0,2024-11-16,Queens,83,23,640.54,27.849565,5.157391,16.750000
1,2024-11-16,Queens,86,10,565.07,56.507000,16.156000,56.161667
2,2024-11-16,Queens,92,32,800.73,25.022813,5.769063,19.071875
3,2024-11-16,Queens,93,34,1808.47,53.190294,8.561765,18.050980
4,2024-11-16,Queens,95,39,1219.73,31.275128,6.577692,25.561966
...,...,...,...,...,...,...,...,...
84555,2024-02-17,Queens,101,2,47.12,23.560000,6.005000,13.583333
84556,2024-02-17,Queens,102,5,179.20,35.840000,7.384000,28.580000
84557,2024-02-17,Queens,117,6,233.18,38.863333,2.890000,23.244444
84558,2024-02-17,Queens,171,4,119.47,29.867500,6.615000,22.162500


In [5]:
import pandas as pd
silver_taxi_daily_zone["date"] = pd.to_datetime(silver_taxi_daily_zone["date"])
silver_fx_daily["date"] = pd.to_datetime(silver_fx_daily["date"])

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 7, Finished, Available, Finished)

/tmp/ipykernel_7091/2053896666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  silver_fx_daily["date"] = pd.to_datetime(silver_fx_daily["date"])


In [6]:
new_silver_fx_daily = (
    silver_fx_daily
    .sort_values("date")
    .set_index("date")
    .asfreq("D")             
    .ffill()                 
    .reset_index()
)

new_silver_fx_daily

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 8, Finished, Available, Finished)

,date,usd_eur
0,1999-01-04,1.1789
1,1999-01-05,1.1790
2,1999-01-06,1.1743
3,1999-01-07,1.1632
4,1999-01-08,1.1659
...,...,...
9854,2025-12-27,1.1787
9855,2025-12-28,1.1787
9856,2025-12-29,1.1766
9857,2025-12-30,1.1757


In [7]:
taxi_fx = (
    silver_taxi_daily_zone
    .merge(new_silver_fx_daily, left_on="date", right_on="date", how="left")
)

taxi_fx["avg_total_amount_eur"] = (
    taxi_fx["avg_total_amount_usd"] / taxi_fx["usd_eur"]
)

taxi_fx["total_amount_eur"] = (
    taxi_fx["total_revenue_usd"] / taxi_fx["usd_eur"]
)

taxi_fx

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 10, Finished, Available, Finished)

,date,Borough_p,PULocationID,trips_count,total_revenue_usd,avg_total_amount_usd,avg_trip_distance,avg_duration_min,usd_eur,avg_total_amount_eur,total_amount_eur
0,2024-11-16,Queens,83,23,640.54,27.849565,5.157391,16.750000,1.0583,26.315379,605.253709
1,2024-11-16,Queens,86,10,565.07,56.507000,16.156000,56.161667,1.0583,53.394123,533.941226
2,2024-11-16,Queens,92,32,800.73,25.022813,5.769063,19.071875,1.0583,23.644347,756.619106
3,2024-11-16,Queens,93,34,1808.47,53.190294,8.561765,18.050980,1.0583,50.260129,1708.844373
4,2024-11-16,Queens,95,39,1219.73,31.275128,6.577692,25.561966,1.0583,29.552233,1152.537088
...,...,...,...,...,...,...,...,...,...,...,...
84555,2024-02-17,Queens,101,2,47.12,23.560000,6.005000,13.583333,1.0768,21.879643,43.759287
84556,2024-02-17,Queens,102,5,179.20,35.840000,7.384000,28.580000,1.0768,33.283804,166.419019
84557,2024-02-17,Queens,117,6,233.18,38.863333,2.890000,23.244444,1.0768,36.091506,216.549034
84558,2024-02-17,Queens,171,4,119.47,29.867500,6.615000,22.162500,1.0768,27.737277,110.949108


In [8]:
taxi_yearly = silver_taxi_daily_zone.copy()
taxi_yearly["year"] = pd.to_datetime(taxi_yearly["date"]).dt.year
taxi_yearly

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 11, Finished, Available, Finished)

,date,Borough_p,PULocationID,trips_count,total_revenue_usd,avg_total_amount_usd,avg_trip_distance,avg_duration_min,year
0,2024-11-16,Queens,83,23,640.54,27.849565,5.157391,16.750000,2024
1,2024-11-16,Queens,86,10,565.07,56.507000,16.156000,56.161667,2024
2,2024-11-16,Queens,92,32,800.73,25.022813,5.769063,19.071875,2024
3,2024-11-16,Queens,93,34,1808.47,53.190294,8.561765,18.050980,2024
4,2024-11-16,Queens,95,39,1219.73,31.275128,6.577692,25.561966,2024
...,...,...,...,...,...,...,...,...,...
84555,2024-02-17,Queens,101,2,47.12,23.560000,6.005000,13.583333,2024
84556,2024-02-17,Queens,102,5,179.20,35.840000,7.384000,28.580000,2024
84557,2024-02-17,Queens,117,6,233.18,38.863333,2.890000,23.244444,2024
84558,2024-02-17,Queens,171,4,119.47,29.867500,6.615000,22.162500,2024


In [9]:
taxi_gdp = taxi_yearly.merge(
    bronze_worldbank_gdp,
    on="year",
    how="left"
)
taxi_gdp

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 12, Finished, Available, Finished)

,date,Borough_p,PULocationID,trips_count,total_revenue_usd,avg_total_amount_usd,avg_trip_distance,avg_duration_min,year,gdp_usd
0,2024-11-16,Queens,83,23,640.54,27.849565,5.157391,16.750000,2024,28750956130731
1,2024-11-16,Queens,86,10,565.07,56.507000,16.156000,56.161667,2024,28750956130731
2,2024-11-16,Queens,92,32,800.73,25.022813,5.769063,19.071875,2024,28750956130731
3,2024-11-16,Queens,93,34,1808.47,53.190294,8.561765,18.050980,2024,28750956130731
4,2024-11-16,Queens,95,39,1219.73,31.275128,6.577692,25.561966,2024,28750956130731
...,...,...,...,...,...,...,...,...,...,...
84555,2024-02-17,Queens,101,2,47.12,23.560000,6.005000,13.583333,2024,28750956130731
84556,2024-02-17,Queens,102,5,179.20,35.840000,7.384000,28.580000,2024,28750956130731
84557,2024-02-17,Queens,117,6,233.18,38.863333,2.890000,23.244444,2024,28750956130731
84558,2024-02-17,Queens,171,4,119.47,29.867500,6.615000,22.162500,2024,28750956130731


In [10]:
gold_taxi_revenue_usd_eur_daily = taxi_fx.copy()
gold_taxi_with_gdp_context = taxi_gdp.copy()

import pyspark.sql.functions as F

gold_taxi_revenue_usd_eur_daily = spark.createDataFrame(gold_taxi_revenue_usd_eur_daily)
gold_taxi_with_gdp_context = spark.createDataFrame(gold_taxi_with_gdp_context)

gold_taxi_revenue_usd_eur_daily.write.mode("append").saveAsTable("gold_taxi_revenue_usd_eur_daily")
gold_taxi_with_gdp_context.write.mode("append").saveAsTable("gold_taxi_with_gdp_context")

StatementMeta(, a4002131-e478-40b2-9426-948c07398399, 12, Finished, Available, Finished)

In [ ]:
gold_taxi_revenue_usd_eur_daily = taxi_fx.copy()
gold_taxi_with_gdp_context = taxi_gdp.copy()

spark_gold_taxi_revenue_usd_eur_daily = spark.createDataFrame(gold_taxi_revenue_usd_eur_daily)
spark_gold_taxi_with_gdp_context = spark.createDataFrame(gold_taxi_with_gdp_context)

spark_gold_taxi_revenue_usd_eur_daily.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_taxi_revenue_usd_eur_daily")

spark_gold_taxi_with_gdp_context.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_taxi_with_gdp_context")

print("Done process")

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 14, Finished, Available, Finished)

Done process


In [ ]:
df = spark.sql("SELECT * FROM Itransition_project.dbo.gold_taxi_revenue_usd_eur_daily LIMIT 1000")
display(df)

StatementMeta(, 092dd8eb-3a66-4a1c-9707-b1c01d5a5637, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cfffb440-5c28-461f-aaf9-83c60f6059f4)